In [4]:
import os
import matplotlib.pyplot as plt
from astropy.table import Table

# Define paths
base_path = "../../SIMBA100_fitting_results/v_2500_data"
output_path = os.path.join(base_path, "Plots")
os.makedirs(output_path, exist_ok=True)

# Define plotting functions
def plot_scatter(detections, group_id):
    plt.figure(figsize=(10, 6))
    plt.scatter(detections['v'], detections['N'], alpha=0.7, edgecolor='k')
    plt.xlabel('v (km/s)')
    plt.ylabel('N (log scale)')
    plt.title(f'N vs. v for Group {group_id}')
    plt.grid(alpha=0.3)
    plt.savefig(os.path.join(output_path, f"group_{group_id}_N_vs_v.png"), dpi=300)
    plt.close()

def plot_histogram(detections, column, group_id, bins=30):
    plt.figure(figsize=(10, 6))
    plt.hist(detections[column], bins=bins, alpha=0.7, color='skyblue', edgecolor='k')
    plt.xlabel(column)
    plt.ylabel('Frequency')
    plt.title(f'{column} Distribution for Group {group_id}')
    plt.grid(alpha=0.3)
    plt.savefig(os.path.join(output_path, f"group_{group_id}_{column}_histogram.png"), dpi=300)
    plt.close()

def plot_scatter_with_error(detections, group_id):
    top_10 = detections.nlargest(10, 'N')[['N', 'dN', 'v']]
    print(f"Top 10 N values for Group {group_id}:\n", top_10)
    plt.figure(figsize=(10, 6))
    plt.errorbar(
        detections['v'], detections['N'], yerr=detections['dN'],
        fmt='o', ecolor='red', alpha=0.7, capsize=3, markerfacecolor='blue',
        markeredgecolor='k', linestyle='none'
    )
    plt.xlabel('v (km/s)')
    plt.ylabel('N (log scale)')
    plt.title(f'N vs. v (with errors) for Group {group_id}')
    plt.grid(alpha=0.3)
    plt.savefig(os.path.join(output_path, f"group_{group_id}_N_vs_v_error.png"), dpi=300)
    plt.close()

# Loop through files and generate plots
for file in sorted(os.listdir(base_path)):
    if file.endswith('combined_results.txt'):
        group_id = file.split('_')[1]
        full_path = os.path.join(base_path, file)

        try:
            data = Table.read(full_path, format='ascii.fixed_width', delimiter='|').to_pandas()
            data['N'] = pd.to_numeric(data['N'], errors='coerce')
            data['dN'] = pd.to_numeric(data['dN'], errors='coerce')
            data['v'] = pd.to_numeric(data['v'], errors='coerce')
            detections = data.dropna(subset=['N', 'dN', 'v'])

            plot_scatter(detections, group_id)
            plot_histogram(detections, 'N', group_id)
            plot_histogram(detections, 'v', group_id)
            plot_scatter_with_error(detections, group_id)

        except Exception as e:
            print(f"Failed to process Group {group_id}: {e}")